In [1]:
import time
import os
import sys
import json
import itertools
import numpy as np

from cpymad.madx import Madx

In [2]:
sys.path.append("../")

import Models
print(Models.__file__)

/home/dylan/ThesisWorkspace/Tracking/ThinLens/Models.py


setup Mad-X

In [3]:
output = True
madx = Madx(stdout=False)
madx.options.echo = output 
madx.options.warn = output 
madx.options.info = output 

# specify beam
assert madx.command.beam(mass=18.798, charge=7, exn=1.258e-6, eyn=2.005e-6, gamma=1.012291786)  # from Adrian

# select lattice
madx.call("/dev/shm/newSIS18.seq")

# set quadrupole strenghts
k1f = 0.3525911342676681
k1d = -0.3388671731064351
k1f_support = 0.0

madx.input("""
k1f={};
k1d={};
k1f_support={};
""".format(k1f, k1d, k1f_support))

# activate sequence
sequence = "SIS18" 

assert madx.command.use(sequence=sequence)

check tunes

In [4]:
twiss = madx.command.twiss()
print("q1: {:.2f}, q2: {:.2f}".format(madx.table.summ["q1"][0], madx.table.summ["q2"][0]))

q1: 4.20, q2: 3.40


make thin and check tunes again

In [5]:
slices = 8

madx.command.select(sequence=sequence, flag="makethin", slice_=slices)
madx.command.makethin(sequence=sequence, style="simple", makedipedge=True)
assert madx.command.use(sequence=sequence)

# tunes
twiss = madx.command.twiss()
print("q1: {:.2f}, q2: {:.2f}".format(madx.table.summ["q1"][0], madx.table.summ["q2"][0]))

q1: 4.21, q2: 3.41


matching routine example

In [6]:
matchCommand = """
    match, sequence={};
    global, sequence={}, q1={}, q2={};
    vary, name=k1f, step=0.00001;
    vary, name=k1d, step=0.00001;
    lmdif, calls=500, tolerance=1.0e-10;
    endmatch;
    """.format(sequence, sequence, 4.2, 3.4)

assert madx.input(matchCommand) 

check final parameter values

In [7]:
twiss = madx.command.twiss()
print(madx.table.summ["q1"], madx.table.summ["q2"])

print("k1f: {:.2f}, k1d: {:.2f}".format(madx.globals["k1f"], madx.globals["k1d"]))

[4.20000001] [3.4]
k1f: 0.35, k1d: -0.34


load thin lens model

In [13]:
model = Models.SIS18_Lattice_minimal(k1f=k1f, k1d=k1d, k1f_support=k1f_support,
                                      slices=slices, quadSliceMultiplicity=1)

# model = Models.SIS18_Lattice_minimal(k1f=madx.globals["k1f"], k1d=madx.globals["k1d"],
#                                      k1f_support=madx.globals["k1f_support"],
#                                       slices=slices, quadSliceMultiplicity=1)

print(model.getTunes())

[0.19235964459840524, 0.4174420321208551]


load model into Mad-X

In [18]:
madModel = Madx(stdout=False)

# specify beam
assert madModel.command.beam(mass=18.798, charge=7, exn=1.258e-6, eyn=2.005e-6, gamma=1.012291786)  # from Adrian

# select lattice
madModel.input(model.thinMultipoleMadX())

sequence = "SIS18" 
assert madModel.command.use(sequence=sequence)

In [19]:
# check tunes
assert madModel.command.twiss()
print("q1: {:.2f}, q2: {:.2f}".format(madModel.table.summ["q1"][0], madModel.table.summ["q2"][0]))

q1: 4.19, q2: 3.42
